# Exploration de l'API de la SNCF

Voici le lien de la documentation : https://doc.navitia.io/#getting-started

On commence par importer nos modules utiles pour cette exploration : 

In [9]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime


In [2]:
# Charger la clé API
load_dotenv("../.env")
API_KEY = os.getenv("SNCF_API_KEY")
HEADERS = {"Authorization": API_KEY}

## 📍 Recherche de gares

In [4]:
def chercher_gare(nom):
    url = f"https://api.sncf.com/v1/coverage/sncf/places?q={nom}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()["places"]
    else:
        print("Erreur:", response.status_code)
        return []



In [5]:
# Recherche
resultats = chercher_gare("Nice Riquier")

# Affichage lisible
for r in resultats:
    if r["embedded_type"] == "stop_area":
        sa = r["stop_area"]
        print("Nom :", sa["name"])
        print("ID  :", sa["id"])
        print("Coordonnées :", sa["coord"])
        print("-" * 30)

Nom : Nice Riquier
ID  : stop_area:SNCF:87756353
Coordonnées : {'lon': '7.290175', 'lat': '43.705554'}
------------------------------


In [7]:
# Recherche
resultats = chercher_gare("Monaco")

# Affichage lisible
for r in resultats:
    if r["embedded_type"] == "stop_area":
        sa = r["stop_area"]
        print("Nom :", sa["name"])
        print("ID  :", sa["id"])
        print("Coordonnées :", sa["coord"])
        print("-" * 30)

Nom : Monaco Monte Carlo
ID  : stop_area:SNCF:87756403
Coordonnées : {'lon': '7.419747', 'lat': '43.738414'}
------------------------------


## 🚆 Recherche de trajets entre 2 gares

In [14]:
# Appel API SNCF
from_id = "stop_area:SNCF:87756353"  # Nice Riquier
to_id = "stop_area:SNCF:87756403"    # Monaco Monte Carlo

url = f"https://api.sncf.com/v1/coverage/sncf/journeys?from={from_id}&to={to_id}&count=1"
response = requests.get(url, headers=HEADERS)
journey = response.json()["journeys"][0]  # un seul trajet

# Transformer en un DataFrame avec clé/valeur pour lecture complète
df = pd.json_normalize(journey, sep='_')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
df.T  # transposé pour lecture verticale

0
duration                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [12]:
def afficher_trajets(journeys):
    data = []
    for j in journeys:
        dep = datetime.strptime(j["departure_date_time"], "%Y%m%dT%H%M%S")
        arr = datetime.strptime(j["arrival_date_time"], "%Y%m%dT%H%M%S")
        duration = (arr - dep).seconds // 60
        delay = j.get("durations", {}).get("departure_delay", 0) or 0
        status = "🟢 À l'heure" if delay == 0 else f"⚠️ Retard de {delay // 60} min"
        data.append({
            "Départ": dep.strftime("%H:%M"),
            "Arrivée": arr.strftime("%H:%M"),
            "Durée (min)": duration,
            "Statut": status
        })

    return pd.DataFrame(data)

In [15]:
# ⚙️ Appel API
from_id = "stop_area:SNCF:87756353"  # Nice Riquier
to_id = "stop_area:SNCF:87756403"    # Monaco Monte Carlo

url = f"https://api.sncf.com/v1/coverage/sncf/journeys?from={from_id}&to={to_id}&count=8"
response = requests.get(url, headers=HEADERS)
journeys = response.json()["journeys"]

# 📊 Affichage
df = afficher_trajets(journeys)
df

,Départ,Arrivée,Durée (min),Statut
0,05:54,06:13,19,🟢 À l'heure
1,06:27,06:45,18,🟢 À l'heure
2,06:55,07:14,19,🟢 À l'heure
3,07:08,07:27,19,🟢 À l'heure
4,07:25,07:44,19,🟢 À l'heure
5,07:38,07:57,19,🟢 À l'heure
6,07:54,08:13,19,🟢 À l'heure
7,08:07,08:26,19,🟢 À l'heure


## ⏱️ Exploration des retards

In [16]:
for j in journeys:
    print("Départ :", j["departure_date_time"])
    print("Arrivée :", j["arrival_date_time"])
    delay = j.get("durations", {}).get("departure_delay", 0)
    print("Retard (sec) :", delay)
    print("------------")

Départ : 20250624T055400
Arrivée : 20250624T061300
Retard (sec) : 0
------------
Départ : 20250624T062700
Arrivée : 20250624T064500
Retard (sec) : 0
------------
Départ : 20250624T065500
Arrivée : 20250624T071400
Retard (sec) : 0
------------
Départ : 20250624T070800
Arrivée : 20250624T072700
Retard (sec) : 0
------------
Départ : 20250624T072500
Arrivée : 20250624T074400
Retard (sec) : 0
------------
Départ : 20250624T073800
Arrivée : 20250624T075700
Retard (sec) : 0
------------
Départ : 20250624T075400
Arrivée : 20250624T081300
Retard (sec) : 0
------------
Départ : 20250624T080700
Arrivée : 20250624T082600
Retard (sec) : 0
------------


## 🚨 Exploration des disruptions (perturbations)

In [17]:
url = "https://api.sncf.com/v1/coverage/sncf/disruptions?count=3"
response = requests.get(url, headers=HEADERS)
disruptions = response.json()["disruptions"]
disruptions[0]  # aperçu

{'id': 'f38545d3-448a-4d15-b7a8-c062f065debe',
 'disruption_id': 'f38545d3-448a-4d15-b7a8-c062f065debe',
 'impact_id': 'f38545d3-448a-4d15-b7a8-c062f065debe',
 'application_periods': [{'begin': '20250622T081700',
   'end': '20250622T095700'}],
 'status': 'past',
 'updated_at': '20250623T082342',
 'cause': '',
 'severity': {'name': 'trip delayed',
  'effect': 'SIGNIFICANT_DELAYS',
  'color': '#000000',
  'priority': 42},
 'messages': [{'text': 'Conditions de départ non réunies',
   'channel': {'content_type': '',
    'id': 'rt',
    'name': 'rt',
    'types': ['web', 'mobile']}}],
 'impacted_objects': [{'pt_object': {'id': 'SNCF:2025-06-22:871839:1187:Train',
    'name': 'SNCF:2025-06-22:871839:1187:Train',
    'quality': 0,
    'trip': {'id': 'SNCF:2025-06-22:871839:1187:Train', 'name': '871839'},
    'embedded_type': 'trip'},
   'impacted_stops': [{'stop_point': {'id': 'stop_point:SNCF:87586008:Train',
      'name': 'Agen',
      'label': 'Agen (Agen)',
      'coord': {'lon': '0.62086